# Project 2: Subsistence Diets

In [1]:
apikey = "E6haYn1Fyu8cDrLwizdiVB0pPh1FqIlqdEze5lfw" 

In [3]:
# %pip install pandas

import pandas as pd
import numpy as np

### [A] Dietary Reference Intakes Function

Write a function that takes as arguments the characteristics of a person (e.g., age, sex) and returns a `pandas.Series' of Dietary Reference Intakes (DRI's) or "Recommended Daily Allowances" (RDA) of a variety of nutrients appropriate for your population of interest.

In [ ]:
rda = pd.read_csv("rda.csv", index_col = 0)

options = ['Child_1_3', 'Female_4_8', 'Male_4_8', 'Female_9_13', 'Male_9_13', 'Female_14_18', 'Male_14_18', 'Female_19_30', 'Male_19_30', 'Female_31_50', 'Male_31_50', 'Female_51U', 'Male_51U']

bmin = rda.loc[rda['Constraint Type'].isin(['RDA', 'AI'])]
bmax = rda.loc[rda['Constraint Type'].isin(['UL'])]

,Nutrient Type,Unit,Constraint Type,Child_1_3,Female_4_8,Male_4_8,Female_9_13,Male_9_13,Female_14_18,Male_14_18,Female_19_30,Male_19_30,Female_31_50,Male_31_50,Female_51U,Male_51U
Nutrient,,,,,,,,,,,,,,,,
Sodium_max,Mineral,mg,UL,1500.0,1900.0,1900.0,2200.0,2200.0,2300.0,2300.0,2300.0,2300.0,2300.0,2300.0,2300.0,2300.0
Energy_max,Macro,kcal,UL,2500.0,2500.0,2500.0,2800.0,3000.0,3100.0,3100.0,3100.0,3100.0,3100.0,3100.0,3100.0,3100.0


In [20]:
def dietary_ref_intake(age = 20,sex = "Female", data = rda):
    """Takes in age (integer) and sex (string), and returns a Series of dietary reference intakes for the chosen population, you can optionally use a different data frame, the min or max RDAs"""

    if age <= 3:
        col = 'Child_1_3'
    if sex in ["M", "male", "m"]:
        sex = "Male"
    elif sex in ["F", "f", "female"]:
        sex = "Female"
    if age <= 3:
        col = 'Child_1_3'
    elif age >= 51:
        col = sex + "_51U" 
    else:
        age_ranges = [(4,8),(9,13),(14,18),(19,30),(31,50),(51,100)]
        for age_range in age_ranges:
            if age >= age_range[0] and age <= age_range[1]:
                col = sex + '_' + str(age_range[0]) + '_' + str(age_range[1])
    return pd.Series(data[col])  

#### Examples

In [21]:
dietary_ref_intake(age=22,sex='M')

Nutrient
Energy            2400.0
Protein             56.0
Carbohydrate       130.0
Dietary Fiber       33.6
Linoleic Acid       17.0
Linolenic Acid       1.6
Calcium           1000.0
Iron                 8.0
Magnesium          400.0
Phosphorus         700.0
Potassium         4700.0
Sodium_max        2300.0
Zinc                11.0
Copper               0.9
Selenium            55.0
Vitamin A          900.0
Vitamin E           15.0
Vitamin D           15.0
Vitamin C           90.0
Thiamin              1.2
Riboflavin           1.3
Niacin              16.0
Vitamin B6           1.3
Vitamin B12          2.4
Choline            550.0
Vitamin K          120.0
Folate             400.0
Energy_max        3100.0
Name: Male_19_30, dtype: float64

In [28]:
dietary_ref_intake(age=80,sex='F', data = bmax)

Nutrient
Sodium_max    2300.0
Energy_max    3100.0
Name: Female_51U, dtype: float64

### [A] Data on Prices for Different Foods

Construct a google spreadsheet of the prices of different food products for each diet (frozen food diet, meat diet, fresh food diet, liquid diet, and canned-food diet)